In [58]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [76]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [83]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [98]:
#result page under search term medicaid for news that is filtered further to encompass a recent year's worth of articles
url = 'https://washingtonstatestandard.com/?sfid=3156&_sf_s=medicaid&post_date=04012023+04292024'
response = requests.get(url)
response

<Response [200]>

In [99]:
#parse the content so it is easy to navigate and filter out what is wanted
soup = BeautifulSoup(response.text, 'html.parser')
soup

<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots"/>
<meta content="v1.12.4" name="sn_version"/>
<!-- This site is optimized with the Yoast SEO plugin v22.1 - https://yoast.com/wordpress/plugins/seo/ -->
<meta content="en_US" property="og:locale"/>
<meta content="website" property="og:type"/>
<meta content="Washington State Standard" property="og:site_name"/>
<meta content="https://washingtonstatestandard.com/wp-content/uploads/2023/04/Washington-State-Standard-social-share.png" property="og:image"/>
<meta content="1200" property="og:image:width"/>
<meta content="630" property="og:image:height"/>
<meta content="image/png" property="og:image:type"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="@WAStateStandard" name="twitter:site"/>
<script class="yoast-schema-graph" t

In [100]:

df = pd.DataFrame(columns=['Title', 'Author', 'Date'])

while url:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    archive_cards = soup.find_all('h3', class_='archiveCardTitle')
    data = []
    for card in archive_cards:
        title = card.parent.text.strip()
        
        author = card.find_next('a', class_='author')
        authorn = author.text.strip() if author else None

        author_span = card.find_next('span', class_='archiveBylineAuthor')
        datet = author_span.find_next_sibling('span', class_='archiveByline').text.strip() if author_span else None

        data.append({'Title': title, 'Author': authorn, 'Date': datet})

    df = pd.concat([df, pd.DataFrame(data)], ignore_index=True)

    next_button = soup.find('a', class_='next')
    if next_button:
        url = next_button['href']
    else:
        break
df




,Title,Author,Date
0,"Upwards of 16,000 immigrants on track to enrol...",Grace Deng,"- Friday April 26, 2024"
1,WA not meeting Medicaid lead testing requireme...,Grace Deng,"- Friday January 5, 2024"
2,Hospitals plead with Congress to avert $8 bill...,Ashley Murray,"- Sunday September 17, 2023"
3,"Despite federal warnings, red and blue states ...",Nada Hassanein,"- Sunday August 27, 2023"
4,WA expands Medicaid coverage for people nearin...,Grace Deng,"- Tuesday July 25, 2023"
5,"As states hunt for new voters, Massachusetts a...",Zachary Roth,"- Thursday July 6, 2023"
6,Loss of federal protection in Idaho spurs preg...,Kelcie Moseley-Morris,"- Wednesday April 24, 2024"
7,New Washington law will allow traffic cameras ...,Jerry Cornfield,"- Tuesday March 26, 2024"
8,Massive $1.2 trillion spending package that wo...,Jennifer Shutt,"- Thursday March 21, 2024"
9,Meet the WA Republican lawmaker who ‘sounds li...,Grace Deng,"- Wednesday March 20, 2024"


In [101]:
from sqlalchemy import create_engine


user = 'admin'
password = 'isba4715'
host = 'isba-dev-01.cjwugys6yj0u.us-east-1.rds.amazonaws.com'
port_number = '3306'
database_name = 'HPolProject'


connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port_number}/{database_name}'

eng = create_engine(connection_string)

#I am changing the name and the df{} whenever applicable to not make duplicates of the code
tnam = 'ArticlesforMedicaidWA'
df.to_sql(tnam, eng, if_exists='replace', index=False)

57